In [17]:
import tensorflow as tf

In [18]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import pickle
import numpy as np
import spacy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding, LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
import string

In [7]:
aem = '../tmpanne/fullsample/w2v_model_nr_7_window_10_size_300_negsample_15.txt'

In [8]:
path_to_data = "../tmpanne/AEM_data/dataset_vermeer.pkl"

In [9]:
df = pd.read_pickle(path_to_data)

In [15]:
def encodeY(Y):
    '''create one-hot (dummies) for output, see also https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
    encode class values as integers
    '''
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    dummy_y = tf.keras.utils.to_categorical(encoded_Y)
    return dummy_y

In [20]:
X_train, X_test, y_train, y_test = train_test_split([t.translate(str.maketrans('', '', string.punctuation)) for t in df['text']], encodeY(df['topic'].map(int)), test_size = 0.2)

In [22]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_train)

In [23]:
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(X_train)
# pad sequences
max_length = max([len(s.split()) for s in X_train])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define training labels
#ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])

In [24]:
# HIER OVER NADENKEN
encoded_docs = tokenizer.texts_to_sequences(X_test)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [25]:
#Xtrain[0]

array([ 599, 1208,  950, ...,    0,    0,    0], dtype=int32)

In [5]:
embeddings_index = {}
with open(aem) as f:
    numberofwordvectors, dimensions = [int(e) for e in next(f).split()]
    for line in tqdm(f):
        values = line.split()
        embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
        
       # word = values[0]
       # coefs = np.asarray(values[1:], dtype='float32')
      #  embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))
print('Should be {} vectors with {} dimensions'.format(numberofwordvectors, dimensions))

Found 1055100 word vectors.
Should be 1055100 vectors with 300 dimensions


In [88]:
#test = [e for e in ][10]

In [96]:
#sum([np.isnan(vector.any()) for vector in embeddings_index.values()])

0

In [111]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    words_not_found = 0 
    total_words = 0
    DEBUG_lijstmetwoorden = []
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, 300))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in tqdm(vocab.items()):
        e = embedding.get(word, None)
        if e is not None:   # if we do not find the word, we do not want to replace anything but leave the zero's
            weight_matrix[i] = e
            total_words+=1
        else:
            words_not_found+=1
            DEBUG_lijstmetwoorden.append(word)
    print('Weight matrix created. For {} out of {} words, we did not have any embedding.'.format(words_not_found, total_words))
    return DEBUG_lijstmetwoorden, weight_matrix

In [112]:
missingwords, embedding_vectors = get_weight_matrix(embeddings_index, tokenizer.word_index)

100%|██████████| 90058/90058 [00:00<00:00, 243771.26it/s]

Weight matrix created. For 20235 out of 69823 words, we did not have any embedding.


In [133]:
embedding_vectors

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 4.85866785, -2.4100337 ,  0.33942223, ...,  2.51087785,
         1.81039345,  2.77537751],
       [-3.1060853 ,  3.63483071,  1.44617176, ...,  1.62585211,
        -4.65819836,  1.23772573],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.82667762, -0.38899323,  0.09383333, ..., -0.09508716,
        -0.02855732, -0.44373342],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [79]:
len(embedding_vectors), len(Xtrain)

(90059, 2788)

In [114]:
embedding_layer = Embedding(len(tokenizer.word_index)+1, 300, weights=[embedding_vectors], input_length=max_length, trainable=False)

W0730 15:32:01.339829 139652578797312 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [119]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(4, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 36494, 300)        27017700  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 36490, 128)        192128    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 18245, 128)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2335360)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 9341444   
Total params: 36,551,272
Trainable params: 9,533,572
Non-trainable params: 27,017,700
_________________________________________________________________
None


In [136]:
# alternatief model
numberoflabels = 4
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 4, activation='relu'))
model.add(MaxPooling1D(4))
model.add(MaxPooling1D(4))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=numberoflabels, activation='softmax'))   # voor twee categorien sigmoid, voor 1 tanh
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 36494, 300)        27017700  
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 36491, 128)        153728    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 9122, 128)         0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 2280, 128)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 291840)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                18677824  
_________________________________________________________________
dense_9 (Dense)              (None, 4)                

In [137]:
VALIDATION_SIZE=200

model.fit(Xtrain[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(Xtrain[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))

Train on 2588 samples, validate on 200 samples
Epoch 1/3
2588/2588 [==============================] - 403s 156ms/sample - loss: 3.3999 - acc: 0.5359 - val_loss: 1.3414 - val_acc: 0.5750
Epoch 2/3
2588/2588 [==============================] - 427s 165ms/sample - loss: 0.9264 - acc: 0.6341 - val_loss: 1.1430 - val_acc: 0.6250
Epoch 3/3
2588/2588 [==============================] - 405s 156ms/sample - loss: 0.6547 - acc: 0.7291 - val_loss: 1.0814 - val_acc: 0.7000


In [139]:
loss, acc = model.evaluate(Xtest, y_test, verbose=True)
print('Test Accuracy: %f' % (acc*100))

698/698 [==============================] - 32s 46ms/sample - loss: 0.7930 - acc: 0.7221
Test Accuracy: 72.206306


In [125]:
# compile network

# fit network
model.fit(Xtrain, y_train, epochs=3, verbose=True)
# evaluate
loss, acc = model.evaluate(Xtest, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Epoch 1/3
 224/2788 [=>............................] - ETA: 6:29 - loss: 3.3766 - acc: 0.2545

KeyboardInterrupt: 

In [9]:
# embeddings_index['man'] - embeddings_index['vrouw']

In [ ]:
embedding_matrix = np.zeros((N_FEATURES + 1, dimensions))
for i,word in enumerate(vectorizer.get_feature_names()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector